# Supply Chain Management Project

**Data Import and Setup**
- **Action:** Imported necessary libraries (pandas, math) to handle data manipulation
and mathematical rounding operations.

In [1]:
import pandas as pd
import math

**Loading Datasets**
- **Action:** Loaded five key Excel datasets required for the audit:

1. **Courier Rates:** The agreed-upon rate card for shipping.

2. **Invoice:** The bill provided by the courier company containing charged amounts.

3. **SKU Master:** Weight details for individual products (renamed for clarity).

4. **Pincode Zones:** Mapping of warehouse and customer pincodes to delivery zones.

5. **Order Report:** List of SKUs inside each order ID.

In [2]:
rates = pd.read_excel("Courier Company - Rates.xlsx")
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
sku_master = pd.read_excel("Company X - SKU Master.xlsx").rename(columns={'Weight (g)': 'Weight_g'})
pincode_zones = pd.read_excel("Company X - Pincode Zones.xlsx")
order_report = pd.read_excel("Company X - Order Report.xlsx").rename(columns={'ExternOrderNo': 'Order ID'})

**Weight Calculation**

- **Action:**

- Merged the Order Report with the SKU Master to retrieve the weight of each item.

- Grouped items by Order ID to calculate the total weight per order in Kilograms.

- Logic Applied: Shipping charges are based on weight slabs. A new column,
Weight_Slab_X, was created by rounding the total weight of each order up to the
nearest 0.5 KG (e.g., 0.6 KG becomes 1.0 KG).

In [3]:
order_weights = order_report.merge(sku_master, on='SKU').groupby('Order ID')['Weight_g'].sum().div(1000).reset_index()
order_weights.columns = ['Order ID', 'Total_Weight_X_KG']
order_weights['Weight_Slab_X'] = order_weights['Total_Weight_X_KG'].apply(lambda w: math.ceil(w * 2) / 2)

**Zone Identification**
- **Action:**

- Merged the Invoice data with the Pincode Zones data.

- Established the delivery zone (e.g., Zone A, B, C) based on the Warehouse and Customer
pincodes. This is crucial as rates vary significantly by zone.

In [4]:
invoice_zones = invoice.merge(pincode_zones, on=['Warehouse Pincode', 'Customer Pincode'], how='left').rename(columns={'Zone_y': 'Zone_X', 'Zone_x': 'Zone_Courier'})

**Data Consolidation**
- **Action:** Created a master dataframe (df) by merging the invoice data (with zones) and the
calculated order weights. This creates a single view containing all parameters needed to
calculate the price: Zone, Weight Slab, and Shipment Type.

In [5]:
df = invoice_zones.merge(order_weights, on='Order ID')

**Charge Recalculation**
- **Action:** Defined and applied a custom pricing function (calc_charge) to determine the
Expected Charge for every order.

- **Logic:**

- Fetched fixed and additional rates for the specific Zone from the rate card.

- Calculated charges for the first 0.5 KG and any additional weight multiples.

- Applied specific logic for Forward shipments versus RTO (Return to Origin) shipments.

- Computed the Difference: Expected Charge (Internal calculation) minus Billing
Amount (Courier's charge).

In [6]:
def calc_charge(row):
    fwd_fixed = rates.loc[0, f'fwd_{row["Zone_X"]}_fixed']
    fwd_add = rates.loc[0, f'fwd_{row["Zone_X"]}_additional']
    rto_fixed = rates.loc[0, f'rto_{row["Zone_X"]}_fixed']
    rto_add = rates.loc[0, f'rto_{row["Zone_X"]}_additional']
    
    add_weight = max(0, (row['Weight_Slab_X'] - 0.5) / 0.5)
    charge = fwd_fixed + (add_weight * fwd_add)
    if 'RTO' in row['Type of Shipment']:
        charge += rto_fixed + (add_weight * rto_add)
    return charge

df['Expected_Charge'] = df.apply(calc_charge, axis=1)
df['Difference'] = df['Expected_Charge'] - df['Billing Amount (Rs.)']

**Final Summary & Insights**
- **Action:** Categorized orders based on the calculated difference to determine if they were
charged correctly, overcharged, or undercharged.

**Results:**

- **Correctly Charged:** 19 orders were billed correctly (Difference < ₹0.05), totaling
₹1,483.2.

- **Overcharged:** The courier company overcharged on 141 orders. The total recoverable
amount found is ₹8,289.9.

- **Undercharged:** 14 orders were undercharged by the courier, amounting to a difference
of ₹491.5.

In [7]:
summary = pd.DataFrame({
    'Category': ['Correct', 'Overcharged', 'Undercharged'],
    'Count': [sum(df['Difference'].abs() < 0.05), sum(df['Difference'] < -0.05), sum(df['Difference'] > 0.05)],
    'Amount': [df.loc[df['Difference'].abs() < 0.05, 'Billing Amount (Rs.)'].sum(), 
               df.loc[df['Difference'] < -0.05, 'Difference'].abs().sum(), 
               df.loc[df['Difference'] > 0.05, 'Difference'].abs().sum()]
})
print(summary)

       Category  Count  Amount
0       Correct     19  1483.2
1   Overcharged    141  8289.9
2  Undercharged     14   491.5


**Conclusion**
The analysis reveals significant billing discrepancies. The vast majority of orders (141 out of 174
analyzed) were overcharged, suggesting a systematic issue in how the courier calculates
weights or zones compared to Company X's internal data. Company X can use this report to claim
a refund or credit of approximately ₹8,290.